In [2]:
import os
#spark imports
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import csv

from pyspark.sql.types import *
from pyspark.sql.functions import format_number, when


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyspark.sql.functions as F

In [3]:
#change RAM allocation, current = 8GB
#change Core allocation, current = 3 Cores

spark = SparkSession.builder.appName("Project - Chicago crime")\
.config("spark.some.config.option", "some-value")\
.config("spark.driver.memory", "5g")\
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/03 14:52:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
crimes_schema = StructType([StructField("ID", StringType(), True),
                            StructField("Case Number", StringType(), True),
                            StructField("Date", StringType(), True ),
                            StructField("Block", StringType(), True),
                            StructField("IUCR", StringType(), True),
                            StructField("Primary Type", StringType(), True  ),
                            StructField("Description", StringType(), True ),
                            StructField("Location Description", StringType(), True ),
                            StructField("Arrest", BooleanType(), True),
                            StructField("Domestic", BooleanType(), True),
                            StructField("Beat", StringType(), True),
                            StructField("District", StringType(), True),
                            StructField("Ward", StringType(), True),
                            StructField("Community Area", StringType(), True),
                            StructField("FBI Code", StringType(), True ),
                            StructField("X Coordinate", DoubleType(), True),
                            StructField("Y Coordinate", DoubleType(), True ),
                            StructField("Year", IntegerType(), True),
                            StructField("Updated On", DateType(), True ),
                            StructField("Latitude", DoubleType(), True),
                            StructField("Longitude", DoubleType(), True),
                            StructField("Location", StringType(), True )
                            ])


In [5]:
dataset = spark.read.csv('/Users/sudiptabanerjee/Documents/Big Data Project/Chicago-crime-analysis/Data-Preprocessing/Crime-1.csv',header = True,schema = crimes_schema)
#dataset.persist()

In [6]:
#dataset.select(["Latitude","Longitude","Year","X Coordinate","Y Coordinate"]).describe().show()

dataset = dataset.drop('ID')
dataset = dataset.drop('Case Number')
dataset = dataset.drop('FBI Code')
dataset = dataset.drop('Updated On')
dataset = dataset.drop('IUCR')# Could be useful - four digit codes that law enforcement agencies use to classify criminal incidents when taking individual reports.
dataset = dataset.drop('X Coordinate')
dataset = dataset.drop('Y Coordinate')
dataset = dataset.drop('Location')
#dataset = dataset.drop('Arrest')
dataset = dataset.drop('Domestic')
dataset = dataset.drop('Beat')
#dataset = dataset.drop('Location Description')
dataset = dataset.drop('Description')
dataset = dataset.drop('District')
#dataset = dataset.drop('Ward')
dataset = dataset.drop('Community Area')
#dataset = dataset.drop('Longitude')
#dataset = dataset.drop('Latitude')
dataset = dataset.drop('Block')

<font size="4">Rows in original dataset(Year 2001-2020)</font>

In [7]:
dataset.count()

999

<font size="4">Dataset chosen of year 2010-2019</font>

In [8]:
dataset=dataset.filter((dataset['Year'] >2009) & (dataset['Year'] !=2020))

In [9]:
dataset.count()

994

<font size="4">Dropped null values in data</font>

In [10]:
dataset=dataset.na.drop()

In [12]:
<font size="4">Dropped values accounted for <1 % of data

SyntaxError: invalid syntax (524475127.py, line 1)

In [13]:
dataset.count()

979

In [14]:
d=dataset.groupby('Primary Type').count()
d.show(50)

+--------------------+-----+
|        Primary Type|count|
+--------------------+-----+
|OFFENSE INVOLVING...|   15|
|CRIMINAL SEXUAL A...|    8|
|            STALKING|    4|
|PUBLIC PEACE VIOL...|    2|
|           OBSCENITY|    1|
|               ARSON|    2|
|   CRIMINAL TRESPASS|   18|
|             ASSAULT|   90|
| MOTOR VEHICLE THEFT|  103|
|               THEFT|  209|
|             BATTERY|  157|
|             ROBBERY|   36|
|            HOMICIDE|    5|
|        PROSTITUTION|    3|
|  DECEPTIVE PRACTICE|   64|
|CONCEALED CARRY L...|    1|
|         SEX OFFENSE|   15|
|     CRIMINAL DAMAGE|  111|
|           NARCOTICS|   12|
|       OTHER OFFENSE|   55|
|          KIDNAPPING|    2|
|            BURGLARY|   35|
|   WEAPONS VIOLATION|   30|
|INTERFERENCE WITH...|    1|
+--------------------+-----+



In [15]:
#dataset=dataset.filter((dataset['Year'] >2009) & (dataset['Year'] !=2020))

In [16]:
dataset_new=dataset.filter((dataset['Primary Type']!='STALKING')&
                             (dataset['Primary Type']!='OBSCENITY')&
                             (dataset['Primary Type']!='NON-CRIMINAL (SUBJECT SPECIFIED)')&
                             (dataset['Primary Type']!='GAMBLING')&
                             (dataset['Primary Type']!='NON - CRIMINAL')&
                             (dataset['Primary Type']!='LIQUOR LAW VIOLATION')&
                             (dataset['Primary Type']!='PUBLIC INDECENCY')&
                             (dataset['Primary Type']!='HUMAN TRAFFICKING')&
                             (dataset['Primary Type']!='INTIMIDATION')&
                             (dataset['Primary Type']!='CONCEALED CARRY LICENSE VIOLATION')&
                             (dataset['Primary Type']!='NON-CRIMINAL')&
                             (dataset['Primary Type']!='OTHER NARCOTIC VIOLATION'))
dataset_new.show(50)                             

+--------------------+--------------------+--------------------+------+----+----+-----------+------------+
|                Date|        Primary Type|Location Description|Arrest|Ward|Year|   Latitude|   Longitude|
+--------------------+--------------------+--------------------+------+----+----+-----------+------------+
|05/24/2021 03:06:...|            HOMICIDE|              STREET|  true|  36|2021|41.91783806|-87.75596897|
|06/26/2021 09:24:...|            HOMICIDE|         PARKING LOT|  true|  50|2021|41.99521944|-87.71335491|
|       11/09/23 7:30|            BURGLARY|           APARTMENT| false|  47|2023|41.95234509|-87.67797506|
|       11/12/23 7:59|             BATTERY|  SMALL RETAIL STORE|  true|   6|2023|41.73775077|-87.60485591|
|      09/06/23 17:00|     CRIMINAL DAMAGE|PARKING LOT / GAR...| false|  42|2023|41.88601806|-87.63393788|
|       07/01/24 0:00|  DECEPTIVE PRACTICE|AIRPORT EXTERIOR ...| false|  41|2024|41.99491395|-87.88193767|
|08/25/2024 08:21:...|            HOM

In [17]:
dataset_new_1 = dataset_new.withColumn("Primary Type", \
                               when((dataset_new["Primary Type"] == 'CRIM SEXUAL ASSAULT') | \
                                    (dataset_new["Primary Type"] == 'PROSTITUTION') ,'SEX OFFENSE').otherwise(dataset_new['Primary Type']))
dataset_new_1=dataset_new_1.withColumn("Primary Type", \
                               when((dataset_new_1["Primary Type"] == 'WEAPONS VIOLATION') | \
                                    (dataset_new_1["Primary Type"] == 'INTERFERENCE WITH PUBLIC OFFICER') ,'PUBLIC PEACE VIOLATION').otherwise(dataset_new_1['Primary Type']))
dataset_new_1=dataset_new_1.withColumn("Primary Type", \
                               when((dataset_new_1["Primary Type"] == 'ARSON') ,'CRIMINAL TRESPASS').otherwise(dataset_new_1['Primary Type']))
dataset_new_1.show()

+--------------------+--------------------+--------------------+------+----+----+-----------+------------+
|                Date|        Primary Type|Location Description|Arrest|Ward|Year|   Latitude|   Longitude|
+--------------------+--------------------+--------------------+------+----+----+-----------+------------+
|05/24/2021 03:06:...|            HOMICIDE|              STREET|  true|  36|2021|41.91783806|-87.75596897|
|06/26/2021 09:24:...|            HOMICIDE|         PARKING LOT|  true|  50|2021|41.99521944|-87.71335491|
|       11/09/23 7:30|            BURGLARY|           APARTMENT| false|  47|2023|41.95234509|-87.67797506|
|       11/12/23 7:59|             BATTERY|  SMALL RETAIL STORE|  true|   6|2023|41.73775077|-87.60485591|
|      09/06/23 17:00|     CRIMINAL DAMAGE|PARKING LOT / GAR...| false|  42|2023|41.88601806|-87.63393788|
|       07/01/24 0:00|  DECEPTIVE PRACTICE|AIRPORT EXTERIOR ...| false|  41|2024|41.99491395|-87.88193767|
|08/25/2024 08:21:...|            HOM

In [18]:
d=dataset_new_1.groupby('Location Description').count()
d.show()

+--------------------+-----+
|Location Description|count|
+--------------------+-----+
|AIRPORT TERMINAL ...|    1|
|VEHICLE - COMMERCIAL|    1|
|RESIDENCE - YARD ...|   16|
|CHA PARKING LOT /...|    3|
|            SIDEWALK|   28|
|OTHER RAILROAD PR...|    1|
|            CAR WASH|    3|
|         CTA STATION|    3|
|            CEMETARY|    1|
|TAVERN / LIQUOR S...|    2|
|PARKING LOT / GAR...|   48|
|RESIDENCE - PORCH...|   16|
|     APPLIANCE STORE|    1|
|           WAREHOUSE|    2|
|AIRPORT EXTERIOR ...|    1|
|AIRPORT TERMINAL ...|    2|
|        CTA PLATFORM|    2|
|                BANK|    3|
|  GROCERY FOOD STORE|    5|
|         CHA GROUNDS|    1|
+--------------------+-----+
only showing top 20 rows



In [19]:
dataset_new_1.repartition(1).write.csv('my_csv2',mode="append",header=True)